In [2]:
import os
import matplotlib.pyplot as plt
import copy
import time


import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision import transforms, utils
from torchvision import datasets, models, transforms
torch.__version__

import pandas as pd
import numpy as np
from skimage import io, transform
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
!unzip '/content/drive/My Drive/Vision_training/catego/ry_train_41K.zip'
data = pd.read_csv('/content/drive/My Drive/Vision_training/category_train_41K.csv')
data = data[data.Category.notna()]
data[:30000].to_csv('./train_data.csv', index=False)
data[30000:36000].to_csv('./val_data.csv', index=False)
data[36000:].to_csv('./test_data.csv', index=False)

In [5]:
data.Category.unique()

array(['Hatchback', 'Jeep', 'Microbus', 'Sedan', 'Coupe'], dtype=object)

In [ ]:
Category = np.array(data.Category.unique())

class MyautoDataset_colors(Dataset):
  def __init__(self, csv_file, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.df = pd.read_csv(csv_file)
        self.transform = transform
        self.category = Category

  def __len__(self):
    return len(self.df)
    # return 4000

  def __getitem__(self, idx):
    if torch.is_tensor(idx):
        idx = idx.tolist()
    path = '/content/category_train_41K/{}_{}.jpg'.format(self.df.iloc[idx].ID, self.df.iloc[idx].img_index)
    image = transform.resize(io.imread(path), (224, 224))/255
    # y = [0]*len(self.colors)
    # y[np.where(self.colors == self.df.iloc[idx].Color)[0][0]] = 1
    y = np.where(self.category == self.df.iloc[idx].Category)[0]
    sample = (torch.Tensor(np.einsum('ijk->kij',image)), torch.Tensor(y).long())

    # if self.transform:
    #     sample = self.transform(sample)

    return sample

In [ ]:
train = MyautoDataset_colors('./train_data.csv', '/content/training_data')
val = MyautoDataset_colors('./val_data.csv', '/content/training_data')
test = MyautoDataset_colors('./test_data.csv', '/content/training_data')
train_loader = DataLoader(train, batch_size=4,shuffle=True, num_workers=4)
val_loader = DataLoader(val, batch_size=4,shuffle=True, num_workers=4)
test_loader = DataLoader(test, batch_size=4,shuffle=True, num_workers=4)
dataloaders = {'train': train_loader, 'val': val_loader, 'test': test_loader}
dataset_sizes = {'train': len(train_loader), 'val': len(val_loader)/2, 'test': len(test_loader)/2}

In [ ]:
class model_inc(nn.Module):
  def __init__(self):
    super(model_inc, self).__init__()
    self.layers = nn.ModuleList()
    self.layers.append(models.resnet18(pretrained=True))
    self.layers.append(nn.Linear(1000, 256)) 
    self.layers.append(nn.Dropout(0.1))
    self.layers.append(nn.Linear(256, 32)) 
    self.layers.append(nn.Sigmoid())
    self.layers.append(nn.Dropout(0.1))
    self.layers.append(nn.Linear(32, len(data.Category.unique())))
    self.layers.append(nn.Softmax())
    
  def forward(self, x):
    for layer in self.layers:
      x = layer(x)
    return x

In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0
            counter = 0
            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                counter += 1
                if counter % 100 == 0:
                  print(counter)
                  if counter % 1000 == 0:
                    print("loss: {}, acc: {}".format(running_loss/counter, running_corrects.double()/counter/4))
                inputs = inputs.to(device)
                labels = labels.to(device)
                labels = labels.reshape((labels.shape[0]))
                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    # torch.max(outputs, dim=1)
                    preds = torch.argmax(outputs, 1)
                    # _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels.reshape((-1,)))

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc/4))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc/8))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [ ]:
criterion = nn.CrossEntropyLoss()
model = model_inc()
# model.load_state_dict(torch.load('/content/drive/My Drive/Myauto_vision/model_category.pt'))
model.to(device)
# Observe that all parameters are being optimized
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/checkpoints/resnet18-5c106cde.pth


In [ ]:
model = train_model(model, criterion, optimizer, exp_lr_scheduler, num_epochs=1)

In [ ]:
torch.save(model.state_dict(), '/content/drive/My Drive/Vision_training/model_category_me.pt')

In [36]:
optimizer = optim.SGD(model.parameters(), lr=0.002, momentum=0.9)
model = train_model(model, criterion, optimizer, exp_lr_scheduler, num_epochs=5)

Epoch 0/4
----------


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  app.launch_new_instance()


100
200
300
400
500
600
700
800
900
1000
loss: 6.140889871120453, acc: 0.3555
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
loss: 5.945825352549553, acc: 0.41575
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
loss: 5.877541170279185, acc: 0.432
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
loss: 5.824293631613255, acc: 0.4454375
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
loss: 5.779605961799621, acc: 0.45625000000000004
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
loss: 5.75442089176178, acc: 0.46275
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
loss: 5.736900224004473, acc: 0.46675000000000005
7100
7200
7300
7400
7500
train Loss: 5.7259 Acc: 0.4696


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:123: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


100
200
300
400
500
600
700
800
900
1000
loss: 5.43276834487915, acc: 0.54125
1100
1200
1300
1400
1500
val Loss: 10.8536 Acc: 1.0920

Epoch 1/4
----------
100
200
300
400
500
600
700
800
900
1000
loss: 5.517669199705124, acc: 0.5185
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
loss: 5.485755552053451, acc: 0.529125
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
loss: 5.504235747734706, acc: 0.5248333333333333
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
loss: 5.505716060638428, acc: 0.5241875
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
loss: 5.49578805732727, acc: 0.5265500000000001
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
loss: 5.4829902236859, acc: 0.5295
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
loss: 5.47615785084452, acc: 0.5308928571428572
7100
7200
7300
7400
7500
train Loss: 5.4750 Acc: 0.5314
100
200
300
400
500
600
700
800
900
1000
loss: 5.273499984264374, acc: 0.579
1100
1200
1300
1400
1500
val Loss: 10.5439 Acc: 1.1633

Epoch 2/4
---------

In [45]:
# optimizer = optim.SGD(model.parameters(), lr=0.002, momentum=0.9)
model = train_model(model, criterion, optimizer, exp_lr_scheduler, num_epochs=2)

Epoch 0/1
----------


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  app.launch_new_instance()


100
200
300
400
500
600
700
800
900
1000
loss: 4.955487236022949, acc: 0.6625
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
loss: 4.904175392866135, acc: 0.6773750000000001
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
loss: 4.912935279130935, acc: 0.6745
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
loss: 4.903576914906502, acc: 0.676875
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
loss: 4.922943782043457, acc: 0.6716500000000001
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
loss: 4.918012624820073, acc: 0.6730416666666666
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
loss: 4.918690876552037, acc: 0.6732142857142858
7100
7200
7300
7400
7500
train Loss: 4.9182 Acc: 0.6735
100
200
300
400
500
600
700
800
900
1000
loss: 4.941720993757248, acc: 0.665
1100
1200
1300
1400
1500
val Loss: 9.8291 Acc: 1.3467

Epoch 1/1
----------
100
200
300
400
500
600
700
800
900
1000
loss: 4.865986109018325, acc: 0.68525
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
loss: 4.8886

In [44]:
torch.save(model.state_dict(), '/content/drive/My Drive/Vision_training/model_category_67.pt')